In [1]:
using JuMP 
using Gurobi
using DelimitedFiles
using NBInclude
const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001

Set parameter Username
Academic license - for non-commercial use only - expires 2023-02-17


1.0e-5

In [3]:
@nbinclude("dataparser.ipynb")
@nbinclude("mip.ipynb")

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
Edge 1 => 3 has 1 as its origin and 3 as its destination
Edge 1 => 16 has 1 as its origin and 16 as its destination
Edge 2 => 4 has 2 as its origin and 4 as its destination
Edge 2 => 14 has 2 as its origin and 14 as its destination
Edge 3 => 4 has 3 as its origin and 4 as its destination
Edge 3 => 14 has 3 as its origin and 14 as its destination
Edge 4 => 6 has 4 as its origin and 6 as its destination
Edge 4 => 13 has 4 as its origin and 13 as its destination
Edge 4 => 16 has 4 as its origin and 16 as its destination
Edge 5 => 1 has 5 as its origin and 1 as its destination
Edge 5 => 2 has 5 as its origin and 2 as its destination
Edge 5 => 4 has 5 as its origin and 4 as its destination
Edge 5 => 6 has 5 as its origin and 6 as its destination
Edge 5 => 8 has 5 as its origin and 8 as its destination
Edge 5 => 10 has 5 as its origin and 10 as its destination
Edge 5 => 11 has 5 as its origin and 11 as its destination
Edge 5 => 13 has 5 as its origin an

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.7/Project.toml`
  No Changes to `~/.julia/environments/v1.7/Manifest.toml`


solve_MIP (generic function with 1 method)

### Lecture des données

In [4]:
data_folder = string(@__DIR__,"/data")
instance_name = "MD-00001-00000010"
filename = joinpath(data_folder, join([instance_name]))
new_inst = Instance(filename, 3, 0)

Instance({16, 47} directed simple Int64 graph, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], [0.0 0.0 … 0.0 1.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16], Int64[], 16, 0, 3, 0, true)

### Résolution du MIP

In [ ]:
result = @timed solve_MIP()

### Branch and price
#### Génération de colonne
On relaxe la formulation du problème comme suit :

$$ 
\underset{c\in \mathcal{C}^\mathcal{l}}{max}\ \  w_c z_c \\
tel \ que \ \  
\sum_{c\in \mathcal{C}^l(i)} z_c \leq 1 \ \ \ \forall i \in V\\
z \in [0,1]^{\mathcal{C}^l}$$

L'ensemble des cycles réalisables peut être très long à énumérer. On commence donc avec un ensemble initial $\mathcal{C}_1^l \subset \mathcal{C}^l $
La solution obtenue avec cette ensemble initial nous donne une borne primale que l'on note $z_p$.

De plus, notre solution final doit être binaire. On réalise donc un arbre de branchement. A la, la contrainte de branchement est $\{z_i \geq 1\} \ \ (respectvement \ \ \{z_i \leq 0\}$

Ainsi, dans l'arbre de branchement, notre problème maître restreint est :


$$ 
\underset{c\in \mathcal{C_k}^\mathcal{l}}{max}\ \  w_c z_c \\
tel \ que \ \  
\sum_{c\in \mathcal{C_k}^l(i)} z_c \leq 1 \ \ \ \forall i \in V\\
z \in [0,1]^{\mathcal{C_k}^l} \bigcap \{z_c \geq 1\} \ \ (respectvement \ \ \{z_c \leq 0\} $$
